# G16 Code for creating demo data

## 0. Load data

In [ ]:
import pandas as pd
import numpy as np
import sys
print(sys.prefix)

In [ ]:
# import data
dir = '../data/raw/'
filename = 'BMMS_overview.xlsx'
df_bridges = pd.read_excel(dir + filename)

In [ ]:
#df_bridges.head()

In [ ]:
# import data
dir = '../data/raw/'
filename = '_roads3.csv' # replace this with the interpolated data
df_roads = pd.read_csv(dir + filename)

In [ ]:
#df_roads.head()

In [ ]:
# print shape of dataframes
print(df_bridges.shape)
print(df_roads.shape)

## 1. Remove duplicates from BMMS_overview.xlsx

In [ ]:
df_bridges_clean = df_bridges.copy()
df_bridges_clean.info()

In [ ]:
# add a new colum 'road_lrp' combines 'road' and 'lrp'
df_bridges_clean['road_lrp'] = df_bridges_clean['road'] + df_bridges_clean['LRPName']

# add a new column 'name_clean' equals to 'name' 
df_bridges_clean['name_clean'] = df_bridges_clean['name']

# add a new column 'road_chainage' equals to 'chainage', but turn float into string, combine 'road' and 'chainage'
df_bridges_clean['road_chainage'] = df_bridges_clean['chainage'].astype(str)
df_bridges_clean['road_chainage'] = df_bridges_clean['road'] + df_bridges_clean['road_chainage']


In [ ]:

#df_bridges_clean.head()

In [ ]:
# lower the letters 
df_bridges_clean['name_clean'] = df_bridges_clean['name_clean'].str.lower()

# delet space 
df_bridges_clean['name_clean'] = df_bridges_clean['name_clean'].str.replace(' ', '')

# delet hypen
df_bridges_clean['name_clean'] = df_bridges_clean['name_clean'].str.replace('-', '')

# delet comma
df_bridges_clean['name_clean'] = df_bridges_clean['name_clean'].str.replace(',', '')

# mark the unknown road name
df_bridges_clean.loc[df_bridges_clean['name_clean'] == '.', 'name_clean'] = 'unknown'

# delet period 
df_bridges_clean['name_clean'] = df_bridges_clean['name_clean'].str.replace('.', '')

In [ ]:
df_bridges_clean.shape

In [ ]:
# delete duplicate in both 'road_lrp' and 'road_chainage' columns
df_bridges_clean = df_bridges_clean.drop_duplicates(subset=['road_lrp','chainage'], keep='first')
df_bridges_clean.shape

In [ ]:
# replace (r) with (right), (l) with (left) in column 'name_clean'
df_bridges_clean.loc[:, 'name_clean'] = df_bridges_clean['name_clean'].str.replace('right', 'r')
df_bridges_clean.loc[:, 'name_clean'] = df_bridges_clean['name_clean'].str.replace('left', 'l')

In [ ]:
# find the duplicated value in 'road_chainage' column, show the unique values in 'name_clean' column
df_bridges_clean[df_bridges_clean.duplicated(subset=['road_chainage'], keep=False)].sort_values(by='road_chainage').name_clean.unique()

In [ ]:
# Function to filter bridges based on left or right (keep left bridge), and conditions (keep worse condition)
def filter_bridges(df):
    df = df.copy()
    
    # Condition priority (higher = worse condition)
    condition_priority = {'A': 1, 'B': 2, 'C': 3, 'D': 4}
    df['condition_priority'] = df['condition'].map(condition_priority)

    # Group by 'chainage'
    grouped = df.groupby('road_chainage')

    rows_to_keep = []

    for _, group in grouped:
        if len(group) == 1:  # If only one row exists for this chainage, keep it
            rows_to_keep.append(group.index[0])
            continue

        # Identify left and right bridges
        left_bridge = group[group['name_clean'].str.contains('(l)', na=False, regex=False)]
        right_bridge = group[group['name_clean'].str.contains('(r)', na=False, regex=False)]

        if not left_bridge.empty and not right_bridge.empty:
            # If both left and right exist, keep only the left bridge
            rows_to_keep.append(left_bridge.index[0])
        else: 
            # If two (r) or two (l) exist, or none of them exist, keep the one with the worst condition
            worst_bridge = group.sort_values(by='condition_priority', ascending=False).iloc[0]
            rows_to_keep.append(worst_bridge.name)

    # Keep only selected rows
    return df.loc[rows_to_keep].drop(columns=['condition_priority']).reset_index(drop=True)



In [ ]:
# Apply function
df_bridges_clean = filter_bridges(df_bridges_clean)
df_bridges_clean.shape

In [ ]:
# check the duplicated value in 'chainage' column
df_bridges_clean[df_bridges_clean.duplicated(subset=['road_chainage'], keep=False)].sort_values(by=['road_lrp'])

## 2. Remove duplicates from _roads3.csv

In [ ]:
df_roads_clean = df_roads.copy()
df_roads_clean.info()

In [ ]:
# add a new column 'road_chainage' equals to 'chainage', but turn float into string, combine 'road' and 'chainage'
df_roads_clean['road_chainage'] = df_roads_clean['chainage'].astype(str)
df_roads_clean['road_chainage'] = df_roads_clean['road'] + df_roads_clean['road_chainage']

# add a new colum 'road_lrp' combines 'road' and 'lrp'
df_roads_clean['road_lrp'] = df_roads_clean['road'] + df_roads_clean['lrp']


In [ ]:
#df_roads_clean.head()

In [ ]:
# find the duplicated value in 'road_lrp' column
df_roads_clean[df_roads_clean.duplicated(subset=['road_lrp'], keep=False)].sort_values(by='road_lrp')


In [ ]:
# check rows between index 27330 and 27347
df_roads_clean.loc[27330:27347]


In [ ]:
# manually change the lrp in row 27331 to 'LRP028', road_lrp to 'Z2002LRP028'
df_roads_clean.loc[27331, 'lrp'] = 'LRP028'
df_roads_clean.loc[27331, 'road_lrp'] = 'Z2002LRP028'

In [ ]:
# clean the duplicated road_lrp, as all the bridge information come from BMMW_overview.xlsx
df_roads_clean = df_roads_clean.drop_duplicates(subset=['road_lrp'], keep='first')

In [ ]:
# check the duplicated value in 'road_lrp' column
df_roads_clean[df_roads_clean.duplicated(subset=['road_lrp'], keep=False)].sort_values(by='road_lrp')

## 3. Combine data from BMMS_overview.xlsx & _roads3.csv

In [ ]:
# change the column names of the bridges dataframe: LRPName -> lrp
df_bridges_clean.rename(columns={'LRPName': 'lrp'}, inplace=True)


In [ ]:
# add a column named 'model_type' to the df_bridges dataframe and fill it with 'bridge'
df_bridges_clean['model_type'] = 'bridge'

# add a column named 'model_type' to the df_roads dataframe and fill it with 'link';
# but if the last character in the column 'lrp' isn't number, fill it with 'bridge', except for 'S' and 'E'
df_roads_clean['model_type'] = 'link'
df_roads_clean.loc[df_roads['lrp'].str[-1].str.isnumeric() == False, 'model_type'] = 'bridge'
df_roads_clean.loc[df_roads['lrp'].str[-1] == 'S', 'model_type'] = 'link'
df_roads_clean.loc[df_roads['lrp'].str[-1] == 'E', 'model_type'] = 'link'


In [ ]:
#df_roads

In [ ]:
# make a copy of the df_bridges dataframe and name it df_bridges_original
df_bridges_original = df_bridges_clean.copy()

# merge the df_bridges and df_roads dataframes
df_concat = pd.concat([df_bridges_clean, df_roads_clean])

In [ ]:
#df_concat.head()

In [ ]:
# sort the df_concat dataframe, groupedby 'road', by 'chainage'
df_concat = df_concat.sort_values(by=['road', 'chainage'])

In [ ]:
#df_concat.head()

## 4. Build the demo dataframe

In [ ]:
# if the 'model_type' is 'bridge' and the 'condition' is NaN, drop the row; Keep the rows with 'model_type' as 'link'
df_concat = df_concat[~((df_concat['model_type'] == 'bridge') & (df_concat['condition'].isna())) | (df_concat['model_type'] == 'link')]

In [ ]:
# Initialize an empty list to store the rows of the new DataFrame
new_rows = []

# Iterate over each unique road in df_concat
for road in df_concat['road'].unique():
    road_rows = df_concat[df_concat['road'] == road].reset_index(drop=True)
    for i, row in road_rows.iterrows():
        if i == 0:
            model_type = 'source'
        elif i == len(road_rows) - 1:
            model_type = 'sink'
        else:
            model_type = row['model_type']
        
        length = 0 if i == 0 else row['chainage'] - road_rows.loc[i-1, 'chainage']
        new_row = {
            'road': row['road'],
            'id': f"{row['road']}_{i}",
            'model_type': model_type,
            'name': row['name'],
            'lat': row['lat'],
            'lon': row['lon'],
            'length': length,
            'condition': row['condition']
        }
        new_rows.append(new_row)

# Convert the list to a DataFrame
df_demo = pd.DataFrame(new_rows)

In [ ]:
df_demo.head()

In [ ]:
# print shape of df_demo
print(df_demo.shape)

## 5. Save demo file as a csv

In [ ]:
# save the df_demo dataframe to a csv file
dir = '../data/processed/'
filename = 'demo_100.csv'
df_demo.to_csv(dir + filename, index=False)